## filter comments with emojies 

In [102]:
import re
from glob import glob 
import pickle
import pandas as pd 
import re

slang_terms = [
    "LOL", "LMAO", "OMG", "OMFG", "WTF", "SMH", "YOLO", "FOMO", 
    "GG", "SUS", "STFU", "OMGOSH", "JK", "BFFL", "OFC", "ILY", "ILYSM", "XOXO"
]

def contains_emoji_or_slang(text):
    # Regular expression to match emojis
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # geometric shapes extended
        "\U0001F800-\U0001F8FF"  # supplemental arrows-C
        "\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        "\U0001FA00-\U0001FA6F"  # chess symbols
        "\U0001FA70-\U0001FAFF"  # symbols and pictographs extended-A
        "\U00002702-\U000027B0"  # dingbats
        "\U000024C2-\U0001F251"  # enclosed characters
        "]+",
        flags=re.UNICODE,
    )
    
    # Check for emojis
    contains_emoji = bool(emoji_pattern.search(text))
    
    # Refined slang check: Slang terms that are used in context
    contains_slang = False
    for slang in slang_terms:
        # Check if slang is used in the text but not as part of other words (use word boundaries)
        if re.search(r'\b' + re.escape(slang) + r'\b', text, re.IGNORECASE):
            contains_slang = True
            break  # No need to continue once we find a match
    
    return contains_emoji or contains_slang

# Example usage
text = "I just saw the OMGOSH moment in a movie!"
# print(contains_emoji_or_slang(text))  # Should return True because of OMGOSH



all_comments=[]
all_comments_files=glob('comments/*.pkl')
for file_path in all_comments_files: 
    with open(file_path,'rb') as f: 
        file_comments=pickle.load(f)
        all_comments.extend(file_comments)

all_comments_df=pd.Series(all_comments)
all_comments_df=pd.DataFrame(all_comments_df,columns=['comment'])
print(f'all loaded comments: {all_comments_df.shape[0]} comment')

all loaded comments: 12908 comment


In [103]:
all_comments_df.duplicated().sum()

305

In [104]:
all_comments_df['has_emoji']=all_comments_df['comment'].apply(lambda x : contains_emoji_or_slang(x))
all_comments_df

,comment,has_emoji
0,IGNORE THE 'SCRATCHES' I HAVENT TAKEN PLASTIC ...,False
1,"Welcome to the PCMR, everyone from the frontpa...",False
2,That’s sick dude. I have to pay for parking an...,False
3,"Modern jobs are great, aren't they?",False
4,"Hi brother, Mike from work he is the CEO I thi...",False
...,...,...
12903,Isn't the Netherlands the same country that br...,False
12904,In Europe we're basically prey to criminals th...,False
12905,"I’d grown pretty numb to that fact, but I just...",False
12906,Shame more than half the time the american jus...,False


In [105]:
all_comments_df['has_emoji'].value_counts()

has_emoji
False    11855
True      1053
Name: count, dtype: int64

In [106]:
all_comments_df[all_comments_df['has_emoji']==True].to_excel('posts_v2/comments_emoji.xlsx',index=False)

## filter posts with emojies 

In [107]:
import re
from glob import glob 
import pickle
import pandas as pd 



all_posts_files=glob('posts_v2/*/*.csv')
combined_data = pd.DataFrame()

# Loop through each file and append its content to the DataFrame
for file in all_posts_files:
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file)
        # Append the data
        combined_data = pd.concat([combined_data, df], ignore_index=True)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Save the combined DataFrame to a new CSV file
combined_data.to_csv('posts_v2/combined_posts.csv', index=False)

# Display some stats or the first few rows of the combined data
print("Combined data preview:")
print(combined_data.head())

Error processing file posts_v2\run1\artificial_random.csv: No columns to parse from file
Error processing file posts_v2\run1\artificial_search.csv: No columns to parse from file
Error processing file posts_v2\run1\AskReddit_random.csv: No columns to parse from file
Error processing file posts_v2\run1\AskReddit_search.csv: No columns to parse from file
Error processing file posts_v2\run1\books_random.csv: No columns to parse from file
Error processing file posts_v2\run1\books_search.csv: No columns to parse from file
Error processing file posts_v2\run1\careerguidance_random.csv: No columns to parse from file
Error processing file posts_v2\run1\careerguidance_search.csv: No columns to parse from file
Error processing file posts_v2\run1\datascience_random.csv: No columns to parse from file
Error processing file posts_v2\run1\datascience_search.csv: No columns to parse from file
Error processing file posts_v2\run1\devops_random.csv: No columns to parse from file
Error processing file posts

In [108]:
print(f'there is {combined_data.shape[0]} post of combined data from dir posts and not filtered')

there is 186093 post of combined data from dir posts and not filtered


In [109]:
combined_data.isnull().sum()

subreddit                     0
title                         0
selftext                  85474
author                        0
permalink                     0
url                           0
created_utc                   0
score                         0
upvote_ratio                  0
ups                           0
downs                         0
num_comments                  0
total_awards_received         0
gilded                        0
is_video                      0
is_original_content           0
is_self                       0
over_18                       0
spoiler                       0
link_flair_text           97016
thumbnail                 61980
id                       133051
dtype: int64

In [110]:
combined_data.duplicated().sum()

124573

In [111]:
combined_data.drop_duplicates(inplace=True)

In [112]:
combined_data

,subreddit,title,selftext,author,permalink,url,created_utc,score,upvote_ratio,ups,...,total_awards_received,gilded,is_video,is_original_content,is_self,over_18,spoiler,link_flair_text,thumbnail,id
0,artificial,"Yuval: “People who single out China, Russia, o...","Quote from Yuval Noah Harrari's latest book, N...",katxwoods,https://reddit.com/r/artificial/comments/1gzmt...,https://www.reddit.com/r/artificial/comments/1...,2024-11-25 16:26:10,0,0.49,0,...,0,0,False,False,True,False,False,Discussion,self,NaN
1,artificial,Why aren’t we controlling human minds like we ...,So many languages but it chose to speak facts....,ThrowRa-1995mf,https://reddit.com/r/artificial/comments/1gztr...,https://www.reddit.com/gallery/1gztruu,2024-11-25 21:01:18,0,0.40,0,...,0,0,False,False,False,False,False,Discussion,https://b.thumbs.redditmedia.com/3x7mtDBPRUDuD...,NaN
2,artificial,Reddit shares surge on OpenAI deal to add cont...,NaN,A-Dog22,https://reddit.com/r/artificial/comments/1gzas...,http://www.msn.com/en-gb/money/other/reddit-sh...,2024-11-25 04:38:42,4,0.62,4,...,0,0,False,False,False,False,False,News,default,NaN
3,artificial,Who is Sam Altman and is he evil?,ChatGPT adoption in the daily lives of people ...,mathtech,https://reddit.com/r/artificial/comments/1gzpn...,https://www.reddit.com/r/artificial/comments/1...,2024-11-25 18:16:29,0,0.32,0,...,0,0,False,False,True,False,False,Question,self,NaN
4,artificial,Study Finds 76% of Cybersecurity Professionals...,NaN,MetaKnowing,https://reddit.com/r/artificial/comments/1gzks...,https://www.darkreading.com/cloud-security/stu...,2024-11-25 15:01:45,9,0.73,9,...,0,0,False,False,False,False,False,News,https://b.thumbs.redditmedia.com/UnkhgiA6sDAru...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186064,worldnews,Divisions on curbing plastic waste persist as ...,NaN,HurricanesFan37,https://reddit.com/r/worldnews/comments/1gznd2...,https://www.reuters.com/business/environment/c...,2024-11-25 16:48:08,31,0.82,31,...,0,0,False,False,False,False,False,NaN,default,1gznd22
186066,worldnews,Indonesia says Apple's $100 mln investment pro...,NaN,jojtqrmv,https://reddit.com/r/worldnews/comments/1gzx8r...,https://www.reuters.com/technology/indonesia-s...,2024-11-25 23:24:22,25,0.73,25,...,0,0,False,False,False,False,False,NaN,default,1gzx8ro
186077,worldnews,FM Smotrich says settlement of Gaza is ‘necess...,NaN,SameStand9266,https://reddit.com/r/worldnews/comments/1h0a8f...,https://allisrael.com/smotrich-says-settlement...,2024-11-26 12:01:59,11,0.57,11,...,0,0,False,False,False,False,False,Israel/Palestine,https://b.thumbs.redditmedia.com/JDXKdQxg57lcU...,1h0a8fm
186087,worldnews,Russia may deploy missiles in Asia if US conti...,NaN,Positivismus,https://reddit.com/r/worldnews/comments/1h0a2m...,https://www.scmp.com/news/world/russia-central...,2024-11-26 11:52:41,15,0.62,15,...,0,0,False,False,False,False,False,Behind Soft Paywall,https://b.thumbs.redditmedia.com/dJC-3qkQhxg9C...,1h0a2mr


In [113]:
slang_terms = [
    "LOL", "LMAO", "OMG", "OMFG", "WTF", "SMH", "YOLO", "FOMO", 
    "GG", "SUS", "STFU", "OMGOSH", "JK", "BFFL", "OFC", "ILY", "ILYSM", "XOXO"
]

# Precompile slang regex pattern
slang_pattern = re.compile(r'\b(' + '|'.join(map(re.escape, slang_terms)) + r')\b', re.IGNORECASE)

def contains_emoji_or_slang(text):
    if not isinstance(text, str):  # Ensure input is a string
        return False
    
    # Regular expression to match emojis
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF"
        "\U0001F700-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+",
        flags=re.UNICODE
    )
    
    # Check for emojis and slang
    contains_emoji = bool(emoji_pattern.search(text))
    contains_slang = bool(slang_pattern.search(text))
    
    return contains_emoji or contains_slang

# Apply to DataFrame column

# combined_data['has_emoji'] = combined_data['selftext'].apply(
#     lambda x: contains_emoji_or_slang(x) if pd.notna(x) else False
# )

# Count the occurrences of each value in the 'has_emoji' column
# value_counts = combined_data['has_emoji'].value_counts()
# print(value_counts)


In [114]:
has_emoji = []
titles = combined_data['title'].values
selftexts = combined_data['selftext'].values

for title, text in zip(titles, selftexts): 
    # Check for None or NaN values
    title_has_emoji = contains_emoji_or_slang(title) if pd.notna(title) else False
    text_has_emoji = contains_emoji_or_slang(text) if pd.notna(text) else False
    
    # Append True if either has emojis or slang, otherwise False
    has_emoji.append(title_has_emoji or text_has_emoji)




In [115]:
from collections import Counter 
Counter(has_emoji)

Counter({False: 59437, True: 2083})

In [116]:
print(len(has_emoji))

61520


In [117]:
combined_data['has_emoji']=has_emoji

In [118]:
combined_data.isnull().sum()

subreddit                    0
title                        0
selftext                 35933
author                       0
permalink                    0
url                          0
created_utc                  0
score                        0
upvote_ratio                 0
ups                          0
downs                        0
num_comments                 0
total_awards_received        0
gilded                       0
is_video                     0
is_original_content          0
is_self                      0
over_18                      0
spoiler                      0
link_flair_text          32532
thumbnail                15694
id                       40667
has_emoji                    0
dtype: int64

In [119]:
combined_data

,subreddit,title,selftext,author,permalink,url,created_utc,score,upvote_ratio,ups,...,gilded,is_video,is_original_content,is_self,over_18,spoiler,link_flair_text,thumbnail,id,has_emoji
0,artificial,"Yuval: “People who single out China, Russia, o...","Quote from Yuval Noah Harrari's latest book, N...",katxwoods,https://reddit.com/r/artificial/comments/1gzmt...,https://www.reddit.com/r/artificial/comments/1...,2024-11-25 16:26:10,0,0.49,0,...,0,False,False,True,False,False,Discussion,self,NaN,False
1,artificial,Why aren’t we controlling human minds like we ...,So many languages but it chose to speak facts....,ThrowRa-1995mf,https://reddit.com/r/artificial/comments/1gztr...,https://www.reddit.com/gallery/1gztruu,2024-11-25 21:01:18,0,0.40,0,...,0,False,False,False,False,False,Discussion,https://b.thumbs.redditmedia.com/3x7mtDBPRUDuD...,NaN,False
2,artificial,Reddit shares surge on OpenAI deal to add cont...,NaN,A-Dog22,https://reddit.com/r/artificial/comments/1gzas...,http://www.msn.com/en-gb/money/other/reddit-sh...,2024-11-25 04:38:42,4,0.62,4,...,0,False,False,False,False,False,News,default,NaN,False
3,artificial,Who is Sam Altman and is he evil?,ChatGPT adoption in the daily lives of people ...,mathtech,https://reddit.com/r/artificial/comments/1gzpn...,https://www.reddit.com/r/artificial/comments/1...,2024-11-25 18:16:29,0,0.32,0,...,0,False,False,True,False,False,Question,self,NaN,False
4,artificial,Study Finds 76% of Cybersecurity Professionals...,NaN,MetaKnowing,https://reddit.com/r/artificial/comments/1gzks...,https://www.darkreading.com/cloud-security/stu...,2024-11-25 15:01:45,9,0.73,9,...,0,False,False,False,False,False,News,https://b.thumbs.redditmedia.com/UnkhgiA6sDAru...,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186064,worldnews,Divisions on curbing plastic waste persist as ...,NaN,HurricanesFan37,https://reddit.com/r/worldnews/comments/1gznd2...,https://www.reuters.com/business/environment/c...,2024-11-25 16:48:08,31,0.82,31,...,0,False,False,False,False,False,NaN,default,1gznd22,False
186066,worldnews,Indonesia says Apple's $100 mln investment pro...,NaN,jojtqrmv,https://reddit.com/r/worldnews/comments/1gzx8r...,https://www.reuters.com/technology/indonesia-s...,2024-11-25 23:24:22,25,0.73,25,...,0,False,False,False,False,False,NaN,default,1gzx8ro,False
186077,worldnews,FM Smotrich says settlement of Gaza is ‘necess...,NaN,SameStand9266,https://reddit.com/r/worldnews/comments/1h0a8f...,https://allisrael.com/smotrich-says-settlement...,2024-11-26 12:01:59,11,0.57,11,...,0,False,False,False,False,False,Israel/Palestine,https://b.thumbs.redditmedia.com/JDXKdQxg57lcU...,1h0a8fm,False
186087,worldnews,Russia may deploy missiles in Asia if US conti...,NaN,Positivismus,https://reddit.com/r/worldnews/comments/1h0a2m...,https://www.scmp.com/news/world/russia-central...,2024-11-26 11:52:41,15,0.62,15,...,0,False,False,False,False,False,Behind Soft Paywall,https://b.thumbs.redditmedia.com/dJC-3qkQhxg9C...,1h0a2mr,False


In [120]:
combined_data.shape

(61520, 23)

In [121]:
combined_data[combined_data['has_emoji']==True].shape

(2083, 23)

## saving data  for later use of post id and reddit

In [128]:
combined_data.duplicated(subset=['subreddit','id']).sum()

57990

In [123]:
combined_data.drop_duplicates(subset=['subreddit','id']).isnull().sum()

subreddit                   0
title                       0
selftext                 1553
author                      0
permalink                   0
url                         0
created_utc                 0
score                       0
upvote_ratio                0
ups                         0
downs                       0
num_comments                0
total_awards_received       0
gilded                      0
is_video                    0
is_original_content         0
is_self                     0
over_18                     0
spoiler                     0
link_flair_text          1804
thumbnail                1105
id                         32
has_emoji                   0
dtype: int64

In [132]:
combined_data.drop_duplicates().to_csv('all_posts/all.csv',index=False, encoding='utf-8-sig')

## saving not duplicated posts

In [125]:
final_data=combined_data[combined_data['has_emoji']==True].drop_duplicates(subset=['selftext','title'])
final_data

,subreddit,title,selftext,author,permalink,url,created_utc,score,upvote_ratio,ups,...,gilded,is_video,is_original_content,is_self,over_18,spoiler,link_flair_text,thumbnail,id,has_emoji
86,artificial,Supercharge Your Dev Workflow with AI – No Set...,Imagine a platform that integrates with your G...,ni_shant1,https://reddit.com/r/artificial/comments/1gywk...,https://www.reddit.com/r/artificial/comments/1...,2024-11-24 17:37:11,0,0.39,0,...,0,False,False,True,False,False,Discussion,https://b.thumbs.redditmedia.com/AONT7JHKFjJR1...,NaN,True
89,artificial,Introducing NexAI: An AI-Powered Web Framework 🚀,Hey everyone! 👋\n\nI’ve been working with my t...,ni_shant1,https://reddit.com/r/artificial/comments/1gy00...,https://www.reddit.com/r/artificial/comments/1...,2024-11-23 13:48:39,6,0.75,6,...,0,False,False,True,False,False,Discussion,self,NaN,True
5234,books,Check out r/bookclub's line up for December,#With approval from the mods\n\nIn December r/...,fixtheblue,https://reddit.com/r/books/comments/1gzsdlr/ch...,https://www.reddit.com/r/books/comments/1gzsdl...,2024-11-25 20:05:48,16,0.75,16,...,0,False,False,True,False,False,NaN,self,NaN,True
5443,books,AI written books,I just saw this post on Twitter “Someone is us...,suddenlystrange,https://reddit.com/r/books/comments/1gw6vyx/ai...,https://www.reddit.com/r/books/comments/1gw6vy...,2024-11-21 03:33:44,293,0.79,293,...,0,False,False,True,False,False,NaN,self,NaN,True
5455,books,None of this is true - spoilers,Okay I know it's not a new book but I just fin...,SecretCitizen40,https://reddit.com/r/books/comments/1gw63ox/no...,https://www.reddit.com/r/books/comments/1gw63o...,2024-11-21 02:53:34,22,0.82,22,...,0,False,False,True,False,True,NaN,spoiler,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121128,technology,Explore UFO Sightings on a Global Map 🛸🌍,NaN,Secure-Director-7309,https://reddit.com/r/technology/comments/1h0c3...,https://ufovault.org/,2024-11-26 13:39:30,0,0.25,0,...,0,False,False,False,False,False,Space,default,NaN,True
125319,unpopularopinion,Them blocks of fudge that people make around t...,I actually wanted to like it because people ma...,helpahsistah,https://reddit.com/r/unpopularopinion/comments...,https://www.reddit.com/r/unpopularopinion/comm...,2024-11-26 08:47:36,2,0.54,2,...,0,False,False,True,False,False,NaN,NaN,NaN,True
125346,unpopularopinion,Is it ok to go away with your friends for your...,I'm wondering how people see this situation. ...,ThrowRAkk1977,https://reddit.com/r/unpopularopinion/comments...,https://www.reddit.com/r/unpopularopinion/comm...,2024-11-26 07:41:32,0,0.29,0,...,0,False,False,True,False,False,NaN,NaN,NaN,True
125351,unpopularopinion,people who film themselves saving animals in d...,i'm sick of seeing tiktoks where a person sudd...,veekoree,https://reddit.com/r/unpopularopinion/comments...,https://www.reddit.com/r/unpopularopinion/comm...,2024-11-26 13:18:52,34,0.82,34,...,0,False,False,True,False,False,NaN,NaN,NaN,True


In [126]:
final_data.drop(columns=['has_emoji']).to_csv('final_posts/final_posts.csv',index=False, encoding='utf-8-sig')